In [ ]:
import os
from pathlib import Path

import pandas as pd
import torch
from src import analysis, util
from src.analysis import evaluation

os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [ ]:
# ---------------------------------------------------------------------
# Configuration
# ---------------------------------------------------------------------
model = "FNO_lhs_var10_plog100_seed9_20251205_155420"

dataset_name_id = "lhs_var10_plog100_seed9"
dataset_name_ood = "lhs_var20_plog100_seed9"

In [ ]:
# ---------------------------------------------------------------------
# Paths
# ---------------------------------------------------------------------
checkpoint_path = Path(f"../data/processed/{model}/best_model_state_dict.pt")

dataset_cases_path_id = Path(f"../../data/raw/{dataset_name_id}/cases")
dataset_cases_path_ood = Path(f"../../data/raw/{dataset_name_ood}/cases") if dataset_name_ood else None

save_root_id = Path(f"../data/processed/{model}/analysis/id")
save_root_ood = Path(f"../data/processed/{model}/analysis/ood") / dataset_name_ood

In [ ]:
def run_or_load_artifacts_evaluation(
    dataset_name: str,
    save_root: Path,
    dataset_path: Path,
) -> tuple[pd.DataFrame, Path]:
    """Load an existing Parquet artifact for the given dataset, or create it if missing."""
    parquet_path = save_root / f"{dataset_name}.parquet"

    if parquet_path.exists():
        print(f"[INFO] Found existing parquet -> loading: {parquet_path}")
        df = pd.read_parquet(parquet_path)
        print(f"[INFO] Loaded df with {len(df)} rows")
        return df, parquet_path

    print(f"[INFO] Creating artifacts for dataset: {dataset_name}")
    print(f"[INFO] save_root:    {save_root}")
    print(f"[INFO] dataset_path: {dataset_path}")
    print(f"[INFO] checkpoint:   {checkpoint_path}")

    model, loader, processor, device = analysis.analysis_interference.load_inference_context(
        dataset_path=dataset_path,
        checkpoint_path=checkpoint_path,
        batch_size=1,
    )

    print("[INFO] Inference context ready, starting artifact generation...")
    df, parquet_path = analysis.analysis_artifacts.generate_artifacts(
        model=model,
        loader=loader,
        processor=processor,
        device=device,
        save_root=save_root,
        dataset_name=dataset_name,
    )

    print("[INFO] Artifact generation done.")
    return df, parquet_path

In [ ]:
# ---------------------------------------------------------------------
# ID artifacts
# ---------------------------------------------------------------------
df_id, parquet_id = run_or_load_artifacts_evaluation(
    dataset_name=dataset_name_id,
    save_root=save_root_id,
    dataset_path=dataset_cases_path_id,
)

# ---------------------------------------------------------------------
# OOD artifacts (optional)
# ---------------------------------------------------------------------
df_ood = None

if dataset_cases_path_ood is not None:
    df_ood, parquet_ood = run_or_load_artifacts_evaluation(
        dataset_name=dataset_name_ood,
        save_root=save_root_ood,
        dataset_path=dataset_cases_path_ood,
    )

In [ ]:
df_eval_id = evaluation.evaluation_dataframe.load_and_build_eval_df(parquet_id)
df_eval_ood = None
if dataset_cases_path_ood is not None:
    df_eval_ood = evaluation.evaluation_dataframe.load_and_build_eval_df(parquet_ood)

In [ ]:
# =====================================================================
# TOGGLE SETUP
# =====================================================================

toggle = util.util_nb.make_toggle_shortcut(df=df_eval_ood, dataset_name=dataset_name_ood)

# =====================================================================
# 3. SAMPLE VIEWER (nur 3-1 für Testing)
# =====================================================================

sample_viewer_plots = [
    toggle("3-1. GT vs Prediction", evaluation.evaluation_plot.evaluation_plot_sample_viewer.plot_sample_prediction_overview),
]

# =====================================================================
# SECTIONS UND TABS (nur Sample Viewer)
# =====================================================================

sections = [
    util.util_nb.make_dropdown_section(sample_viewer_plots),
]

tab_titles = [
    "3. Sample Viewer",
]

# =====================================================================
# FINAL PANEL
# =====================================================================

evaluation_panel = util.util_nb.make_lazy_panel_with_tabs(
    sections,
    tab_titles=tab_titles,
    open_btn_text=f"{dataset_name_id} — Open Sample Viewer",
    close_btn_text="Close",
)

display(evaluation_panel)

In [ ]:
# =====================================================================
# TOGGLE SETUP
# =====================================================================
toggle = util.util_nb.make_toggle_shortcut(
    df=df_eval_id,
    dataset_name=dataset_name_id,
    df_alt=df_eval_ood,
    dataset_name_alt=dataset_name_ood,
)


# =====================================================================
# 1. GLOBAL ERROR ANALYSIS
# =====================================================================
global_error_analysis_plots = [
    toggle("1-1. Global error metrics", evaluation.evaluation_plot.plot_global_error_metrics),
    toggle("1-2. Global error distribution", evaluation.evaluation_plot.plot_error_distribution),
    toggle("1-3. GT vs Prediction", evaluation.evaluation_plot.plot_global_gt_vs_pred),
    toggle("1-4. Mean error maps", evaluation.evaluation_plot.plot_mean_error_maps),
    toggle("1-5. Std error maps", evaluation.evaluation_plot.plot_std_error_maps),
]


# =====================================================================
# 2. PERMEABILITY SENSITIVITY
# =====================================================================
permeability_sensitivity_plots = [
    toggle("2-1. Error vs permeability magnitude", evaluation.evaluation_plot.plot_error_vs_kappa_magnitude),
    toggle("2-2. Error vs anisotropy ratio", evaluation.evaluation_plot.plot_error_vs_anisotropy_ratio),
    toggle("2-3. Error vs mean permeability", evaluation.evaluation_plot.plot_error_vs_mean_kappa),
    toggle("2-4. Error vs permeability gradient", evaluation.evaluation_plot.plot_error_vs_kappa_gradient),
]


# =====================================================================
# 3. SAMPLE VIEWER
# =====================================================================
sample_viewer_plots = [
    toggle("3-1. Sample Viewer — GT / Prediction / Error", evaluation.evaluation_plot.evaluation_plot_sample_viewer.plot_sample_prediction_overview),
    toggle(
        "3-2. Sample Viewer — kappa tensor overlays", evaluation.evaluation_plot.evaluation_plot_sample_viewer.plot_sample_kappa_tensor_with_overlay
    ),
]


# =====================================================================
# 4. ID/OOD COMPARISON
# =====================================================================
id_ood_comparison_plots = None

if df_eval_ood is not None:
    id_ood_comparison_plots = [
        toggle("4-1. ID vs OOD metrics", evaluation.evaluation_plot.plot_id_vs_ood_metrics),
        toggle("4-2. ID vs OOD error distributions", evaluation.evaluation_plot.plot_id_vs_ood_error_distributions),
        toggle("4-3. OOD - ID mean error map", evaluation.evaluation_plot.plot_id_vs_ood_mean_error_difference),
    ]


# =====================================================================
# SECTIONS UND TABS
# =====================================================================
sections = [
    util.util_nb.make_dropdown_section(global_error_analysis_plots, f"{dataset_name_id} — Global Error Analysis"),
    util.util_nb.make_dropdown_section(permeability_sensitivity_plots, f"{dataset_name_id} — Permeability Sensitivity"),
    util.util_nb.make_dropdown_section(sample_viewer_plots, f"{dataset_name_id} — Sample Viewer"),
]

if id_ood_comparison_plots is not None:
    sections.append(util.util_nb.make_dropdown_section(id_ood_comparison_plots, f"{dataset_name_id} vs {dataset_name_ood} — ID/OOD Comparison"))


tab_titles = [
    "1. Global Error Analysis",
    "2. Permeability Sensitivity",
    "3. Sample Viewer",
]

if id_ood_comparison_plots is not None:
    tab_titles.append("4. ID/OOD Comparison")


# =====================================================================
# FINAL PANEL
# =====================================================================
evaluation_panel = util.util_nb.make_lazy_panel_with_tabs(
    sections,
    tab_titles=tab_titles,
    open_btn_text=f"{dataset_name_id} — Open Evaluation",
    close_btn_text="Close",
)

display(evaluation_panel)
